In [22]:
from model import *
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [42]:
from dataLoader import *
from torch.utils.data import DataLoader

labels_path = "../labels/train_labels/train_labels.json"
sample_vids = "../../dissData/train_vids"

video_dataset = VideoDataset(sample_vids, labels_path, transform=None, resize_shape=(256, 256), num_frames=16)

data_loader = DataLoader(video_dataset, batch_size=10, shuffle=True, collate_fn=lambda batch: [data for data in batch if data is not None])

In [43]:
print(len(data_loader))

272


In [46]:
#code for making training labels

import json

with open("../labels/complete_labels.json", "r") as file:
    data1 = json.load(file)

id_labels = data1.keys()

with open("../labels/train_labels/train_data.json", "r") as file:
    data2 = json.load(file)

id_labels_train = data2.keys()

merged_data = {}

for id in id_labels_train:
    if id in id_labels:
        if data1[id][1] > 10:
            data1[id][1] = 10
        merged_data[id] = data1[id]


with open('train_labels.json', 'w') as f:
    json.dump(merged_data, f, indent=4)

In [ ]:
fc = FullyConnected()
out_fc = fc(out)
score_reg = ScoreRegressor()
output = score_reg(out_fc)
print(output.shape)

torch.Size([96, 1])


In [ ]:
print(output[1])

tensor([0.0102], grad_fn=<SelectBackward0>)


In [ ]:
data_loader = DataLoader(video_dataset, batch_size=5, shuffle=True)
eteModel = EndToEndModel(classifier, fc,final_score_regressor=ScoreRegressor())

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.AdamW(eteModel.parameters(), lr=0.00001)

num_epochs = 5

for epoch in range(num_epochs):
    for frames, labels in data_loader:
        frames = frames.to(device)
        frames = frames.permute(0, 2, 1, 3, 4)
        labels = labels.to(device)
        optimizer.zero_grad()

        outputs = eteModel(frames)
        classification_output = outputs['classification']
        final_score_output = outputs['final_score']

        # Calculate individual losses for classification and final score
        classification_loss = criterion(classification_output, labels.float().view(-1, 1))
        final_score_loss = criterion(final_score_output, labels.float().view(-1, 1))

        # Combine the losses with appropriate weights or other logic if needed
        loss = classification_loss + final_score_loss

        loss.backward()
        optimizer.step()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 29491200000 bytes.